In [1]:
import sys
sys.path.append("..")
import tensorflow as tf
import keras
from model.blazepose import BlazePose
from model.mobilenet_v3 import MobileNetV3
# from model.applications.mobilenet_v3 import MobileNetV3Small
from model.measurement_attention_mlp import get_measurement_attention_mlp
# from model.applications.mobilenet_v2 import MobileNetV2
# from model.tensorflow.python.keras.applications.mobilenet_v3 import MobileNetV3Small


In [2]:
config = {
    # common
    'is_two_head_model': True,

    # model
    'input_shape': [256,256,3],
    'batch_size': 4,
    'path_pretrained': None, 
    'type_backbone': "mbnv3", 
    
    # loss
    'type_loss_fn': 'wing',
    
    # data
    'path_classes': "../seg_classes.txt",
    'shuffle': True, 
    'is_normalized': False, 
    'is_with_seg': False, 
    'path_dataset': "C:\\Users\\user\\work\\data\\body_parts_measurement",
    'num_category_bmi': 10,
    'num_category_height': 10,

    # train
    'epochs': 30,
    'eval_term': 2
}

In [3]:
input_shape = config["input_shape"]
batch_size = config["batch_size"]
type_backbone = config["type_backbone"]
is_with_seg = config["is_with_seg"]
num_category_bmi = config["num_category_bmi"]
num_category_height = config["num_category_height"]

In [4]:
measurement_attention_mlp = get_measurement_attention_mlp(batch_size=batch_size, shape_categorical_data=num_category_bmi + num_category_height)

In [5]:
print(measurement_attention_mlp.inputs)
print(measurement_attention_mlp.outputs)

[<KerasTensor: shape=(4, 20) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(4, 32, 32, 1) dtype=float32 (created by layer 'tf.reshape')>, <KerasTensor: shape=(4, 16, 16, 1) dtype=float32 (created by layer 'tf.reshape_1')>, <KerasTensor: shape=(4, 8, 8, 1) dtype=float32 (created by layer 'tf.reshape_2')>]


In [6]:
input_layer = tf.keras.layers.Input(shape=(256,256,3), batch_size=4)

In [7]:
model = MobileNetV3(input_layer=input_layer, type="small", attention_model=measurement_attention_mlp, num_seg_channels=10, num_keypoints=31)

1 (4, 64, 64, 16)
2 (4, 32, 32, 24)
3 (4, 32, 32, 24)
4 (4, 16, 16, 40)
5 (4, 16, 16, 40)
6 (4, 16, 16, 40)
7 (4, 16, 16, 48)
8 (4, 16, 16, 48)
segs KerasTensor(type_spec=TensorSpec(shape=(4, 128, 128, 10), dtype=tf.float32, name=None), name='p_re_lu_2/add:0', description="created by layer 'p_re_lu_2'")
9 (4, 8, 8, 96)
10 (4, 8, 8, 96)
11 (4, 8, 8, 96)
12 (4, 8, 8, 576)
13 (4, 2, 2, 577)
14 (4, 2, 2, 1280)
15 (4, 2, 2, 1000)
Model: "MobileNetV3_small"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(4, 256, 256, 3)]   0                                            
__________________________________________________________________________________________________
ConvBnAct_0 (ConvBnAct)         (4, 128, 128, 16)    512         input_2[0][0]                    
_________________________________________________

In [ ]:
print(model.inputs)
print(model.outputs)

In [ ]:
model.summary()

In [ ]:
model = tf.keras.applications.MobileNetV3Small(include_top=False)

In [ ]:
x = model(input_layer)

In [ ]:
x = tf.keras.layers.DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=True)(model.output)
x = tf.keras.layers.Conv2D(31, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
x = tf.keras.layers.PReLU(alpha_initializer='uniform')(x)
output_layer = tf.squeeze(x)

In [ ]:
model = tf.keras.Model(inputs=model.input, outputs=output_layer)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    layer.trainable=True
    print("layer",layer)

In [ ]:
model.summary()